# **Problem Statement 1**  
### **Filtering the Noise: ML for Trustworthy Location Reviews**  
**Team 3Pandas** *(Tran Ha My, Diane Teo Min Xuan, Ng Yuen Ning)*  

---

## **Problem Statement**  
Design and implement an **ML-based system** to evaluate the **quality** and **relevancy** of Google location reviews. The system should:  

- **Gauge review quality:** Detect spam, advertisements, irrelevant content, and rants from users who have likely never visited the location.  
- **Assess relevancy:** Determine whether the content of a review is genuinely related to the location being reviewed.  
- **Enforce policies:** Automatically flag or filter out reviews that violate the following example policies:  
  - No advertisements or promotional content.  
  - No irrelevant content (e.g., reviews about unrelated topics).  
  - No rants or complaints from users who have not visited the place (can be inferred from content, metadata, or other signals).  

---

## **Motivation & Impact**  
- **For Users:** Increases trust in location-based reviews, leading to better decision-making.  
- **For Businesses:** Ensures fair representation and reduces the impact of malicious or irrelevant reviews.  
- **For Platforms:** Automates moderation, reduces manual workload, and enhances platform credibility.  

---

## **Data Sources**  

| **Data Sources**       | **Details** |
|-------------------------|-------------|
| **Public Datasets**    | - **Google Review Data:** Open datasets containing Google location reviews (e.g., [Google Local Reviews on Kaggle](https://www.kaggle.com/datasets/denizbilginn/google-maps-restaurant-reviews))<br>- **Google Local review data:** [UCSD Public Dataset](https://mcauleylab.ucsd.edu/public_datasets/gdrive/googlelocal/)<br>- **Alternative Sources:** Yelp, TripAdvisor, or other open review datasets for supplementary training. |
| **Student-Crawled Data** | - Students are encouraged to crawl additional reviews from Google Maps (in compliance with Google's terms of service).<br>- **Example:** [Scraping Google Reviews (YouTube)](https://www.youtube.com/watch?v=LYMdZ7W9bWQ) |


### Dependencies

In [1]:
!pip install iterative-stratification
! pip install tldextract
!pip install -q transformers accelerate datasets bitsandbytes peft trl torch tensorboard
!pip install -U "huggingface_hub[cli]"
!pip install -q huggingface_hub
!pip install -q einops
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 2.1 MB/s eta 0:00:00


In [110]:
import yaml
import os
import json

# ! pip install tldextract
import re
import tldextract

from transformers import pipeline
from tqdm import tqdm

# ! pip install tldextract
from textblob import TextBlob
import pandas as pd

import torch
from torch.cuda import amp
from torch.cuda.amp import autocast, GradScaler
from transformers import pipeline
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
from transformers import AutoModelForCausalLM, AutoTokenizer

from sklearn.model_selection import train_test_split
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import numpy as np

import torch
from torch.utils.data import IterableDataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
import torch.nn as nn
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
from torch.optim import AdamW
from sklearn.metrics import precision_recall_fscore_support, average_precision_score

import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import Dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import gc
import psutil
from huggingface_hub import login

from getpass import getpass
import bitsandbytes as bnb
from peft import LoraConfig, get_peft_model, PeftModel

from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import precision_score, recall_score, f1_score

import shutil
from google.colab import files

### 1. Load Data

In [3]:
from google.colab import files

uploaded = files.upload()

Saving all_reviews_with_labels_normalised.csv to all_reviews_with_labels_normalised.csv
Saving synthetic_combined.csv to synthetic_combined.csv


In [4]:
all_reviews = list(uploaded.keys())[0]
synthetic_combined = list(uploaded.keys())[1]

In [53]:
# with open("config.yaml", "r") as f:
#     config = yaml.safe_load(f)

# labeled_input_folder = config['labeled_input']
# synthetic_folder = config['synthetic_folder']

# full_df = pd.read_csv(f'{synthetic_folder}/all_reviews_with_labels_normalised.csv')
# full_df.isnull().sum()

full_df = pd.read_csv(all_reviews)
full_df = full_df.dropna(subset=['rating']).reset_index(drop=True)

print(f"Loaded {all_reviews} with {len(full_df)} rows")
print(full_df.isnull().sum())

Loaded all_reviews_with_labels_normalised.csv with 11667 rows
review_text             0
rating                  0
has_photo               0
author_name             0
user_review_count       0
business_name           0
category                0
source                  0
review_id               0
comprehensive_review    0
is_ad                   0
is_relevant             0
is_rant                 0
is_legit                0
dtype: int64


In [52]:
synthetic_df = pd.read_csv(synthetic_combined)

def s(col):
    return synthetic_df[col].fillna("NA").astype(str).str.strip()

has_photo_str = np.where(synthetic_df["has_photo"].fillna(False), "yes", "no")
MAX_REVIEW_CHARS = 2000
review_text_clean = s("review_text").str.replace(r"\s+", " ", regex=True).str[:MAX_REVIEW_CHARS]

synthetic_df["comprehensive_review"] = (
    "[Business] " + s("business_name") +
    " | [Category] " + s("category") +
    " | [Rating] " + s("rating") +
    " | [Author] " + s("author_name") +
    " | [User Review Count] " + s("user_review_count") +
    " | [Has Photo] " + pd.Series(has_photo_str, index=synthetic_df.index) +
    " | [Source] " + s("source") +
    " | [Review] " + review_text_clean
).str.replace(r"\s+\|\s+\[Review\]\s+NA$", "", regex=True)

print(f"Loaded {synthetic_combined} with {len(synthetic_df)} rows")
print(synthetic_df.isnull().sum())

Loaded synthetic_combined.csv with 714 rows
review_text             0
rating                  0
has_photo               0
author_name             0
user_review_count       0
business_name           0
category                0
source                  0
review_id               0
is_ad                   0
is_rant                 0
is_legit                0
is_relevant             0
comprehensive_review    0
dtype: int64


In [51]:
to_clean_df = full_df.dropna(subset=['review_text', 'is_ad', 'is_relevant', 'is_rant', 'is_legit'])

to_clean_df.head()

,review_text,rating,has_photo,author_name,user_review_count,business_name,category,source,review_id,comprehensive_review,is_ad,is_relevant,is_rant,is_legit
0,Love the convenience of this neighborhood carw...,4.0,False,Doug Schmidt,1.0,"Auto Spa Speedy Wash - Harvester, MO",['Car wash'],google,1001,"[Business] Auto Spa Speedy Wash - Harvester, M...",False,True,False,True
1,"2 bathrooms (for a large 2 story building), 1 ...",2.0,False,Duf Duftopia,1.0,Kmart,"['Discount store', 'Appliance store', 'Baby st...",google,1002,[Business] Kmart | [Category] ['Discount store...,True,True,True,False
2,My favorite pizza shop hands down!,5.0,False,Andrew Phillips,1.0,Papa’s Pizza,"['Pizza restaurant', 'Chicken wings restaurant...",google,1003,[Business] Papa’s Pizza | [Category] ['Pizza r...,False,True,False,True
3,BOTCHED INSTRUMENT REPAIR IS COSTING US HUNDRE...,1.0,False,Julie Heiland,1.0,The Music Place,['Musical instrument store'],google,1004,[Business] The Music Place | [Category] ['Musi...,False,True,True,False
4,Very unprofessional!!!!!,1.0,False,Alan Khasanov,1.0,Park Motor Cars Inc,['Used car dealer'],google,1005,[Business] Park Motor Cars Inc | [Category] ['...,False,True,True,False


### 2. Pre-Process Datafames

##### 2.1 Cleaning Functions

In [46]:
def normalize_whitespace(text):
    return re.sub(r'\s+', ' ', text).strip()

def clean_text(text):
    if pd.isna(text):
        return ""
    text = str(text)
    text = normalize_whitespace(text)
    return text

##### 2.2 Compute Basic Signals

In [47]:
def compute_basic_signals(text):
    url_count = len(re.findall(r'https?://\S+', text))
    phone_count = len(re.findall(r'\+?\d[\d\s-]{7,}\d', text))
    caps_ratio = sum(1 for c in text if c.isupper()) / max(len(text), 1)
    return url_count, phone_count, caps_ratio

##### 2.3 Sentiment Analysis

In [48]:
def add_textblob_sentiment(df, text_col="review_text", positive_threshold=0.9, negative_threshold=-0.9):
    def get_sentiment(text):
        if pd.isna(text) or not isinstance(text, str) or text.strip() == "":
            return 0.0, 0.0
        try:
            analysis = TextBlob(text)
            return analysis.sentiment.polarity, analysis.sentiment.subjectivity
        except Exception:
            return 0.0, 0.0

    sentiment_results = df[text_col].apply(get_sentiment)
    df["sentiment_polarity"], df["sentiment_subjectivity"] = zip(*sentiment_results)

    df["is_extreme_sentiment"] = df["sentiment_polarity"].apply(
        lambda x: 1 if x >= positive_threshold or x <= negative_threshold else 0
    )

    return df

##### Apply to Dataframe

In [49]:
def preprocess_reviews(df):
    df["clean_text"] = df["review_text"].apply(clean_text)
    signals = df["clean_text"].apply(compute_basic_signals)
    df["url_count"], df["phone_count"], df["caps_ratio"] = zip(*signals)
    return df

cleaned_df = preprocess_reviews(to_clean_df)
print(cleaned_df.head())

cleaned_synthetic_df = preprocess_reviews(synthetic_df)
print(cleaned_synthetic_df.head())

                                         review_text  rating  has_photo  \
0  Love the convenience of this neighborhood carw...     4.0      False   
1  2 bathrooms (for a large 2 story building), 1 ...     2.0      False   
2                 My favorite pizza shop hands down!     5.0      False   
3  BOTCHED INSTRUMENT REPAIR IS COSTING US HUNDRE...     1.0      False   
4                           Very unprofessional!!!!!     1.0      False   

       author_name  user_review_count                         business_name  \
0     Doug Schmidt                1.0  Auto Spa Speedy Wash - Harvester, MO   
1     Duf Duftopia                1.0                                 Kmart   
2  Andrew Phillips                1.0                          Papa’s Pizza   
3    Julie Heiland                1.0                       The Music Place   
4    Alan Khasanov                1.0                   Park Motor Cars Inc   

                                            category  source  review_id  \

In [50]:
# # Save as JSON
# output_json_path = os.path.join(labeled_input_folder, "cleaned_df.json")
# cleaned_df.to_json(output_json_path, orient="records", lines=True, force_ascii=False)
# print(f"JSON file saved to: {output_json_path}")

# # Save as Parquet
# output_parquet_path = os.path.join(labeled_input_folder, "cleaned_df.parquet")
# cleaned_df.to_parquet(output_parquet_path, index=False)
# print(f"Parquet file saved to: {output_parquet_path}")

### 3. Train-Test Split with Multi-Label Stratification

In [69]:
meta_cols = ["clean_text", "url_count","phone_count","caps_ratio","rating","has_photo","user_review_count"]
label_cols = ["is_ad","is_relevant","is_rant"]

X = cleaned_df.drop(columns=label_cols)
y = cleaned_df[label_cols].values

mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)
train_val_idx, test_idx = next(mskf.split(X, y))

train_val_df = cleaned_df.iloc[train_val_idx].reset_index(drop=True)
test_df = cleaned_df.iloc[test_idx].reset_index(drop=True)

y_train_val = y[train_val_idx]
train_idx, val_idx = next(mskf.split(train_val_df.drop(columns=label_cols), y_train_val))

train_df_original = train_val_df.iloc[train_idx].reset_index(drop=True)
val_df = train_val_df.iloc[val_idx].reset_index(drop=True)

train_df = pd.concat([train_df_original, cleaned_synthetic_df], ignore_index=True).reset_index(drop=True)

print(f"Training set size: {train_df.shape} (includes synthetic data)")
print(f"Validation set size: {val_df.shape}")
print(f"Test set size: {test_df.shape}")

print(f"Synthetic data in validation set: {val_df['review_id'].isin(cleaned_synthetic_df['review_id']).any()}")
print(f"Synthetic data in test set: {test_df['review_id'].isin(cleaned_synthetic_df['review_id']).any()}")

Training set size: (8181, 18) (includes synthetic data)
Validation set size: (1866, 18)
Test set size: (2334, 18)
Synthetic data in validation set: False
Synthetic data in test set: False


### 4. Tokenisation

In [67]:
def simple_tokenize(text):
    text = str(text).lower()
    tokens = re.findall(r'\b[a-z]+\b', text)
    return tokens

train_df['tokens'] = train_df['clean_text'].apply(simple_tokenize)
test_df['tokens'] = test_df['clean_text'].apply(simple_tokenize)

In [68]:
print(train_df.shape)
print(val_df.shape)
print(test_df.shape)

(8343, 19)
(1907, 18)
(2384, 19)


### Yuen Ning's model

In [86]:
model_name = "Qwen/Qwen1.5-0.5B"  # or "google/gemma-2b"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [93]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3, torch_dtype="auto")
config = LoraConfig(r=8, lora_alpha=32, lora_dropout=0.1)
model = get_peft_model(model, config)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at Qwen/Qwen1.5-0.5B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [94]:
training_args = TrainingArguments(
    output_dir="./qwen_gemma_multilabel",
    # evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    num_train_epochs=3,
    fp16=True,
    report_to="none",
    remove_unused_columns=False # Add this line to ignore unused columns
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = torch.sigmoid(torch.tensor(logits)).numpy()
    preds = (probs > 0.5).astype(int)

    precision = precision_score(labels, preds, average='macro', zero_division=0)
    recall = recall_score(labels, preds, average='macro', zero_division=0)
    f1 = f1_score(labels, preds, average='macro', zero_division=0)

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [95]:
def preprocess(batch):
    return tokenizer(batch["clean_text"], truncation=True, padding="max_length", max_length=512)

def prepare_dataset(df):
    # Apply filtering to the DataFrame directly
    df = df.reset_index(drop=True)
    return Dataset.from_pandas(df[["clean_text"] + label_cols])

# Filter train_df before creating the dataset
train_df_filtered = train_df.drop(index=7026).reset_index(drop=True)

train_dataset = prepare_dataset(train_df_filtered)
val_dataset = prepare_dataset(val_df)

train_dataset = train_dataset.map(preprocess, batched=True)
val_dataset = val_dataset.map(preprocess, batched=True)
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "is_ad", "is_relevant", "is_rant"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "is_ad", "is_relevant", "is_rant"])


Map:   0%|          | 0/8180 [00:00<?, ? examples/s]

Map:   0%|          | 0/1866 [00:00<?, ? examples/s]

In [98]:
def data_collator(batch):
    input_ids = torch.stack([item["input_ids"] for item in batch])
    attention_mask = torch.stack([item["attention_mask"] for item in batch])
    labels = torch.stack([
        torch.tensor([item["is_ad"], item["is_relevant"], item["is_rant"]], dtype=torch.float)
        for item in batch
    ])
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset, # Use the Hugging Face Dataset
    eval_dataset=val_dataset,   # Use the Hugging Face Dataset
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

/tmp/ipython-input-3843213006.py:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [99]:
trainer.train()

Step,Training Loss
500,0.447100
1000,0.187000
1500,0.110400


TrainOutput(global_step=1536, training_loss=0.24450827219213048, metrics={'train_runtime': 2028.9279, 'train_samples_per_second': 12.095, 'train_steps_per_second': 0.757, 'total_flos': 2.330922766565376e+16, 'train_loss': 0.24450827219213048, 'epoch': 3.0})

In [100]:
model.save_pretrained("./qwen_gemma_multilabel_final")
tokenizer.save_pretrained("./qwen_gemma_multilabel_final")

('./qwen_gemma_multilabel_final/tokenizer_config.json',
 './qwen_gemma_multilabel_final/special_tokens_map.json',
 './qwen_gemma_multilabel_final/chat_template.jinja',
 './qwen_gemma_multilabel_final/vocab.json',
 './qwen_gemma_multilabel_final/merges.txt',
 './qwen_gemma_multilabel_final/added_tokens.json',
 './qwen_gemma_multilabel_final/tokenizer.json')

In [103]:
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 0.13003966212272644, 'eval_precision': 0.7716463472962802, 'eval_recall': 0.6625803733996433, 'eval_f1': 0.7074768587565138, 'eval_runtime': 66.6749, 'eval_samples_per_second': 27.987, 'eval_steps_per_second': 3.51, 'epoch': 3.0}


In [111]:
shutil.make_archive("qwen_gemma_multilabel_final", 'zip', "./qwen_gemma_multilabel_final")
files.download("qwen_gemma_multilabel_final.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>